## 03. Строим вопрос-ответного бота по технологии Retrieval-Augmented Generation на LangChain

[LangChain](https://python.langchain.com) - это набирающая популярность библиотека для работы с большими языковыми моделями и для построения конвейеров обработки текстовых данных. В одной библиотеке присутствуют все элементы, которые помогут нам создать вопрос-ответного бота на наборе текстовых данных: вычисление эмбеддингов, запуск больших языковых моделей для генерации текста, поиск по ключу в векторных базах данных и т.д.

Для начала, установим `langchain` и сопутствующие библиотеки.

In [4]:
%pip install -q langchain sentence_transformers lancedb unstructured yandex_chain yandexcloud

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


 37%|███▋      | 2407/6541 [01:00<01:29, 46.02it/s]

## Разбиваем документ на части

Работа Retrieval-Augmented Generation основана на тексте, который мы извлекли из Вики в первом ноутбуке. Если вы пропустили этот этап (например, в рамках мастер-класса), то распакуйте заготовленный текст, выполнив следующую ячейку. Также в этот текст можно добавить распознанный текст с видео, полученный в результате выполнения предыдущего ноутбука.

In [ ]:
!7za x /home/jupyter/datasphere/s3/s3store/mcdata/harry_text.zip

Для работы retrival augmented generation нам необходимо по запросу найти наиболее релевантные фрагменты исходного текста, на основе которых будет формироваться ответ. Для этого нам надо разбить текст на такие фрагменты, по которым мы сможем вычислять эмбеддинг, и которые будут с запасом помещаться во входное окно используемой большой языковой модели.

Для этого можно использовать механизмы фреймворка LangChain - например, `RecursiveCharacterTextSplitter`. Он разбивает текст на перекрывающиеся фрагменты по набору типовых разделителей - абзацы, переводы строк, разделители слов.

> **ВАЖНО**: Перед выполнением ячейки не забудьте установить имя пользователя, которое вы использовали на предыдущем шаге.

Размер `chunk_size` нужно выбирать исходя из нескольких показателей:
* Допустимая длина контекста для эмбеддинг-модели. Yandex GPT Embeddings допускают 2048 токенов, в то время как многие открытые модели HuggingFace имеют длину контекста 512-1024 токена
* Допустимый размер окна контекста большой языковой модели. Если мы хотим использовать в запросе top 3 результатов поиска, то 3 * chunk_size+prompt_size+response_size должно не превышать длины контекста модели.

In [1]:
import langchain
import langchain.document_loaders

user = "shwars"
chunk_size = 1024
chunk_overlap = 50
source_dir = "content"

loader = langchain.document_loaders.DirectoryLoader(
    source_dir, glob="*.txt", show_progress=True, recursive=True
)
splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
fragments = splitter.create_documents([x.page_content for x in loader.load()])
len(fragments)

100%|██████████| 6541/6541 [02:25<00:00, 44.98it/s]


22985

## Вычисляем эмбеддинги для всех фрагментов

Для вычисления эмбеддингов можно взять какую-нибудь модель из репозитория HuggingFace, с поддержкой русского языка. LangChain содержит свои встроенные средства работы с эмбеддингами, и поддерживает модели из HuggingFace: 

In [2]:
embeddings = langchain.embeddings.HuggingFaceEmbeddings(
    model_name="distiluse-base-multilingual-cased-v1"
)
sample_vec = embeddings.embed_query("Hello, world!")
len(sample_vec)

2023-11-29 17:05:22.838316: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


512

Также можно использовать более продвинутую модель [эмбеддингов от Yandex GPT](https://cloud.yandex.ru/docs/yandexgpt/api-ref/Embeddings/). Для работы с ней используем библиотеку [yandex_chain](http://githib.com/yandex-datasphere/yandex-chain], пока соответствующая поддержка не появилась в LangChain. Нам также потребуется `folder_id` в соответствии с вашими данными.

> Не забудьте добавить секрет **api_key** в свой проект DataSphere!> Не забудьте добавить секрет **api_key** в свой проект DataSphere!

In [1]:
import os

api_key = os.environ["api_key"]
folder_id = "b1g6krtrd2vcbunvjpg6"

In [2]:
from yandex_chain import YandexEmbeddings

embeddings = YandexEmbeddings(folder_id=folder_id, api_key=api_key)
vec = embeddings.embed_query("Hello, world!")
len(vec)

256

### Cохраняем эмбеддинги  в векторную БД

Для поиска нам нужно уметь быстро сопоставлять эмбеддинг запроса, и эмбеддинги всех фрагементов наших исходных материалов. Для этого используются векторные базы данных. Для крупных проектов имеет смысл использовать серьезные инструменты, типа [OpenSearch](https://opensearch.org/) (доступный [в виде сервиса в облаке Yandex Cloud](https://cloud.yandex.ru/services/managed-opensearch)), но для нашего примера мы используем небольшую векторную БД [LanceDB](https://lancedb.com/), хранящую индекс в директории на диске.

Первоначально создадим базу данных, добавив туда одну строчку:

In [8]:
import os

import lancedb
from langchain.vectorstores import LanceDB

db_dir = "store"

os.makedirs(db_dir, exist_ok=True)

db = lancedb.connect(db_dir)
table = db.create_table(
    "vector_index",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

Далее проиндексируем все выделенные ранее фрагменты нашей документации, используя реализованный нами выше адаптер для YandexGPT-эмбеддингов. 

> В зависимости от объема текста, эта ячейка может выполняться достаточно длительное время - вспомним про задержку в 1 сек между запросами! Если вы хотите посмотреть на работу бота в рамках мастер-класса - пропустите следующую ячейку, и загрузите готовую базу с эмбеддингами, выполнив ячейку ниже!

In [ ]:
# не выполняйте эту ячейку в рамках мастер-класса - она
# выполняется долго...
db = LanceDB.from_documents(fragments, embeddings, connection=table)

В рамках мастер-класса воспользуемся готовой проиндексированной базой данных:

In [15]:
!rm -rf store
!7za x /home/jupyter/datasphere/s3/s3store/mcdata/vector_store.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel Xeon Processor (Icelake) (606A0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 20501673 bytes (20 MiB)

Extracting archive: /home/jupyter/datasphere/s3/s3store/mcdata/vector_store.zip
--
Path = /home/jupyter/datasphere/s3/s3store/mcdata/vector_store.zip
Type = zip
Physical Size = 20501673

Everything is Ok

Folders: 5
Files: 7
Size:       51797553
Compressed: 20501673


In [3]:
import lancedb
from langchain.vectorstores import LanceDB

db_dir = "store"
db = lancedb.connect(db_dir)
table = db.open_table("vector_index")
vec_store = LanceDB(table, embeddings)

Теперь посмотрим, насколько хорошо находятся фрагменты текста по какому-то запросу:

In [4]:
q = "Какое заклинание помогает левитировать?"

res = vec_store.similarity_search(q)
for x in res:
    print("-" * 40)
    print(x.page_content)

----------------------------------------
1 Волшебники, использовавшие заклинание 2 Этимология 3 Вопрос о нужности заклинания 4 Появления 5 Примечания 6 См. также
----------------------------------------
Парящий торт  Левитационные заклинания — группа заклинаний, главным действием которых является левитация объектов. Известные заклинания[] Левитационные чары Асцендио Арресто моментум Карпе Ретрактум Левиосо Левикорпус Локомотор Эффект[] Заклинания группы Локомотор способны поднимать объекты в воздух (возможно, они гораздо эффективнее, чем Вингардиум Левиоса). Также с помощью этих заклинаний можно с легкостью перемещать предметы, не используя физических сил. Левитационные заклинания отрицают законы гравитации, помогают волшебнику поднимать тяжелые предметы, которые он не поднял бы без помощи магии. Левитационные заклинания можно наложить неодушевлённый предмет и заставить его летать. Например, мётлы. Заклинание «Вингардиум Левиоса» входило в экзаменационный материал на сдачу СОВ по закли

Ещё один полезный интерфейс для поиска текстов - это `Retriever`, убедимся, что он тоже работает:

In [5]:
retriever = vec_store.as_retriever(search_kwargs={"k": 5})
res = retriever.get_relevant_documents(q)
for x in res:
    print(x.page_content)

1 Волшебники, использовавшие заклинание 2 Этимология 3 Вопрос о нужности заклинания 4 Появления 5 Примечания 6 См. также
Парящий торт  Левитационные заклинания — группа заклинаний, главным действием которых является левитация объектов. Известные заклинания[] Левитационные чары Асцендио Арресто моментум Карпе Ретрактум Левиосо Левикорпус Локомотор Эффект[] Заклинания группы Локомотор способны поднимать объекты в воздух (возможно, они гораздо эффективнее, чем Вингардиум Левиоса). Также с помощью этих заклинаний можно с легкостью перемещать предметы, не используя физических сил. Левитационные заклинания отрицают законы гравитации, помогают волшебнику поднимать тяжелые предметы, которые он не поднял бы без помощи магии. Левитационные заклинания можно наложить неодушевлённый предмет и заставить его летать. Например, мётлы. Заклинание «Вингардиум Левиоса» входило в экзаменационный материал на сдачу СОВ по заклинаниям. Заклинание «Акцио» частично тоже связано с левитацией, так как предметы, п

## Подключаемся к Yandex GPT

Для формирования целостного ответа на вопрос пользователей нам потребуется обработать найденные фрагменты текста с помощью большой языковой модели. Фреймворк LangChain поддерживает интеграцию с различными большими языковыми моделями, включая Yandex GPT. 

In [9]:
from langchain.llms import YandexGPT

instructions = """
Ты - дружелюбный чат-бот, отвечающий на вопросы про вселенную Гарри Поттера
"""

llm = YandexGPT(api_key=api_key, instruction_text=instructions)

llm(q)

'Левитация — это способность парить в воздухе. Заклинание Левитации — это заклинание, позволяющее персонажу парить в течение некоторого времени.'

В данном примере мы пока что никак не использовали наши текстовые документы.

## Собираем Retrieval-Augmented Generation

Пришла пора соединить всё вместе и научить бота отвечать на вопросы, подглядывая в наш текстовый корпус. Для этого используем механизм цепочек (*chains*). Основную функциональность реализует `StuffDocumentsChain`, которая делает следующее:

1. Берёт коллекцию документов `input_documents`
1. Каждый из них пропускает через `document_prompt`, и затем объединяет вместе.
1. Данный текст помещается в переменную `document_variable_name`, и передаётся большой языковой модели `llm`

В нашем случае `document_prompt` не будет модицицировать документ, а основной запрос к LLM будет содержать в себе инструкции для Yandex GPT. 

In [14]:
import langchain.chains
import langchain.prompts

# Промпт для обработки документов
document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

# Промпт для языковой модели
document_variable_name = "context"
stuff_prompt_override = """
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
Текст:
-----
{context}
-----
Вопрос:
{query}"""
prompt = langchain.prompts.PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Создаём цепочку
llm_chain = langchain.chains.LLMChain(llm=llm, prompt=prompt)
chain = langchain.chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)
chain.run(input_documents=res, query=q)

'Заклинание, которое помогает левитировать, называется "Вингардиум Левиоса".'

Чтобы ещё более улучшить результат, мы можем использовать хитрый механизм перемешивания документов, таким образом, чтобы наиболее значимые документы были ближе к началу запроса. Также мы оформим все операции в одну функцию `answer`:

In [15]:
from langchain.document_transformers import LongContextReorder

reorderer = LongContextReorder()


def answer(query, reorder=True, print_results=False):
    results = retriever.get_relevant_documents(query)
    if print_results:
        for x in results:
            print(f"{x.page_content}\n--------")
    if reorder:
        results = reorderer.transform_documents(results)
    return chain.run(input_documents=results, query=query)

In [17]:
answer("Что означает Вингардиум Левиоса", print_results=False)

'**Вингардиум Левио́са** (ориг. *Wingardium Levioso*) — заклятие, поднимающее предметы в воздух.'

Можно сравнить результаты, выдаваемые Yandex GPT напрямую, с результатами нашей генерации на основе документов:

In [18]:
def compare(q):
    print(f"Ответ YGPT: {llm(q)}")
    print(f"Ответ бота: {answer(q)}")


compare("Что означает Вингардиум Левиоса")

Ответ YGPT: Вингардиум — это заклинание, используемое для левитации объектов. Левитация — это способность поднимать объект в воздух без использования магии.

Вингардиум левиоса — это заклинание на языке магов, которое используется для левитирования объектов. Это заклинание используется в Гарри Поттере, чтобы поднимать предметы.
Ответ бота: Вингардиум Левиоса — это заклинание левитации. Оно заставляет предметы парить в воздухе.
